## Import Neo4j

In [1]:
from neo4j import GraphDatabase

In [2]:
import pandas as pd

#### Create Connection to Neo4j Database

In [3]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
conn = Neo4jConnection(uri="neo4j://127.0.0.1:7687", user="team", pwd="F0110wTh€M0n€y")

#### Create Pandas Dataframe with Bitcoin Sent and Received

In [5]:
queryTemplate = """
MATCH p=(tx:Transaction)-[S:SENDS|RECEIVES]-(terror:Address{isTerror:TRUE})
RETURN S.value AS Bitcoin, terror.address AS Address, terror.terrorWallet AS WalletID, tx.date AS Date,type(S) AS SendReceive
"""

In [6]:
response = conn.query(queryTemplate, db='neo4j')

In [24]:
resultList = list(map(lambda x : dict(x), response))
resultDF = pd.DataFrame(resultList)

#### Make df compatible to historic BTC data and translate Satoshi to BTC

In [25]:
resultDF.rename({'Date': 'date'}, axis=1, inplace=True)
resultDF["Bitcoin"] = resultDF["Bitcoin"] / 10**8

In [71]:
resultDF.head(5)

,date,Address,WalletID,SendReceive,Bitcoin
0,2019-02-27,12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8,Terror-wallet ID40,SENDS,0.005000
1,2019-03-05,12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8,Terror-wallet ID40,SENDS,0.004943
2,2019-03-05,12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8,Terror-wallet ID40,SENDS,0.004898
3,2019-03-27,12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8,Terror-wallet ID40,SENDS,0.004854
4,2019-03-27,12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8,Terror-wallet ID40,SENDS,0.004780


#### Export df for future use

In [12]:
resultDF.to_csv("./TerrorTransactions2.csv")

#### Import Historic Bitcoin Pricing Data

In [13]:
rates = pd.read_csv('./btcData.csv')

In [14]:
rates['Datum']= pd.to_datetime(rates['Datum']).dt.date
rates.rename({'Datum': 'date'}, axis=1, inplace=True)
rates = rates[['date','Zuletzt']]

#### Make data compatible and join dataframes

In [29]:
resultDF = resultDF[['date', 'Address','WalletID', 'SendReceive', 'Bitcoin']]
resultDF['date'] = resultDF['date'].map(lambda x: x.to_native())

In [72]:
df2 = pd.merge(rates, resultDF, how='inner', on = 'date')
df2.head(3)

,date,Zuletzt,Address,WalletID,SendReceive,Bitcoin
0,2022-03-05,"37.718,0",19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.011114
1,2022-01-05,"38.461,0",19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.012780
2,2022-01-05,"38.461,0",19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,SENDS,0.014974
3,2022-04-30,"37.650,0",19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.006480
4,2022-04-30,"37.650,0",19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.004097


#### Combine data to create column with transacion values in real currency

In [74]:
df2['Zuletzt'] = df2['Zuletzt'].map(lambda x: float(x.replace(",",".").replace(".","",1)))
df2['tx_value'] = df2['Zuletzt']*df2['Bitcoin']

In [75]:
df2.head(3)

,date,Zuletzt,Address,WalletID,SendReceive,Bitcoin,tx_value
0,2022-03-05,37718.0,19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.011114,419.184274
1,2022-01-05,38461.0,19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.012780,491.544657
2,2022-01-05,38461.0,19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,SENDS,0.014974,575.912322
3,2022-04-30,37650.0,19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.006480,243.972000
4,2022-04-30,37650.0,19D1iGzDr7FyAdiy3ZZdxMd6ttHj1kj6WW,Terror-wallet ID11,RECEIVES,0.004097,154.252426


#### Create df with List of WalletIDs

In [51]:
WalletIDs = df2[['WalletID']].drop_duplicates()

#### Split up dataframe into Send and Receive

In [34]:
dfReceived = df2[df2['SendReceive'] == 'RECEIVES']
dfSends = df2[df2['SendReceive'] == 'SENDS']

#### Calculate earnings through bitcoin usage

In [88]:
expense = dfSends['tx_value'].sum()
BTC_remaining = dfReceived['Bitcoin'].sum()-dfSends['Bitcoin'].sum()
income = dfReceived['tx_value'].sum()+BTC_remaining*27844
earnings = income-expense
print("The total earnings sum up to", earnings, "Euro")

The total earnings sum up to -10098453.712999344 Euro


#### Create Dataframe with earnings for each Wallet

In [67]:
WalletEarnings = []
for Wallet in WalletIDs['WalletID']:
    WalletDfReceived = dfReceived[dfReceived['WalletID'] == Wallet]
    WalletDfSends = dfSends[dfSends['WalletID'] == Wallet]
    expense = WalletDfSends['tx_value'].sum()
    BTC_remaining = WalletDfReceived['Bitcoin'].sum()-WalletDfSends['Bitcoin'].sum()
    income = WalletDfReceived['tx_value'].sum()+BTC_remaining*27844
    earnings = income-expense
    WalletEarnings.append([Wallet, earnings, income])

In [89]:
WalletEarningsDF = pd.DataFrame(WalletEarnings, columns=['WalletID', 'Earnings in €','tx_value_Received in €'])

In [91]:
WalletEarningsDF.sort_values(by = ['tx_value_Received in €'], ascending = False)

,WalletID,Earnings in €,tx_value_Received in €
13,Terror-wallet ID34,-1.093227e+07,1.380746e+10
0,Terror-wallet ID11,7.564434e+05,5.255338e+07
8,Terror-wallet ID38,0.000000e+00,3.583432e+05
6,Terror-wallet ID37,3.563300e-01,5.933284e+04
7,Terror-wallet ID39,0.000000e+00,4.099984e+04
24,Terror-wallet ID29,7.258716e+03,1.307930e+04
2,Terror-wallet ID9,-1.231292e+03,5.914644e+03
12,Terror-wallet ID14,6.727675e+02,3.628499e+03
3,Terror-wallet ID31,0.000000e+00,3.579638e+03
15,Terror-wallet ID24,1.082569e+02,2.824912e+03


In [ ]:
conn.close()